In [1]:
from transformers import AutoModelForCausalLM,AutoProcessor,BitsAndBytesConfig
from PIL import Image
import os
from langchain.docstore.document import Document
from tqdm import tqdm
import pickle

In [6]:
bnb = BitsAndBytesConfig(load_in_8bit = True)

In [7]:
processor = AutoProcessor.from_pretrained("microsoft/Phi-3-vision-128k-instruct",trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-vision-128k-instruct",quantization_config = bnb,device_map = "cuda:0",trust_remote_code = True,_attn_implementation = "eager")
tokenizer = processor.tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
user_prompt = '<|user|>\n'
assistant_prompt = '<|assistant|>\n'
prompt_suffix = "<|end|>\n"

In [4]:
pdfs = os.listdir("Hackathon/llamaParseDocs")

In [5]:
pdfs

['Amazon-2021-Annual-Report',
 'NASDAQ_NVDA_2022',
 'NASDAQ_NVDA_2023',
 '2020_alphabet_annual_report',
 'Amazon-2020-Annual-Report',
 'Apple_10-K-Q4-2020',
 'Apple_10-K-Q4-2022',
 'NASDAQ_NVDA_2020',
 'Apple_10-K-Q4-2023',
 'FB_2022',
 'FB_2021-Annual-Report_FB',
 'FB_2023',
 'Amazon-com-Inc-2023-Annual-Report',
 '2022-alphabet-annual-report',
 '2023_alphabet',
 'Apple_10-K-2021',
 'NASDAQ_NVDA_2021',
 'Amazon-2022-Annual-Report',
 '2024_alphabet-10-q-q1-2024',
 'FB_2020-Annual-Report_FB',
 '.ipynb_checkpoints',
 '2021_alphabet_annual_report']

In [6]:
pics = os.listdir("llamaParseDocs/"+pdfs[0])

NameError: name 'pdfs' is not defined

In [5]:
pics[0]

NameError: name 'pics' is not defined

In [29]:
pdfs.remove(".ipynb_checkpoints")

In [41]:
def summarizeTables(link):
    image = Image.open(link)
    if image.size[1] > 1024:
        image = image.resize((image.size[0],1024))
    elif image.size[0] > 1024:
        image = image.resize((1024,image.size[1]))
    prompt = f"""{user_prompt}<|image_1|>/nYou are an Expert Data Analyst.
    Analyse the Table given above and summarize it, extracting any useful insights.
    Pay close attention to any important numbers and figures in the table and bring them to attention too.
    Stay true to the data in the table and do not make up anything.
    {prompt_suffix}{assistant_prompt}"""
    inputs = processor(prompt,image,return_tensors = "pt").to("cuda:0")
    generation_args = { 
    "max_new_tokens": 425, 
    "temperature": 0.2, 
    "do_sample": True, 
    } 
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 
    return response

In [21]:
summarizeTables("llamaParseDocs/"+pdfs[0]+"/"+pics[0])

'The table presents data on long-term debt and commitments from the year ended December 31, spanning from 2022 to 2026, as well as a total for the entire period. The categories include long-term debt principal and interest, operating lease liabilities, finance lease liabilities (including interest), financing obligations (including interest), leases not yet commenced, unconditional purchase obligations, and other commitments.\n\nThe total long-term debt and commitments show a consistent increase over the years, starting at $29,460 in 2022 and rising to $260,690 by 2026. The largest portion of the total is finance lease liabilities, which increase from $8,278 in 2022 to $9,493 in 2026. Leases not yet commenced also show a steady increase, starting at 1,206 in 2022 and reaching 21,571 by 2026.\n\nThe table also includes a column for the year 2023, which is not part of the total. The data for 2023 is as follows:\n\n- Long-term debt principal and interest: $4,852\n- Operating lease liabili

In [42]:
def summarizeFigures(link):
    image = Image.open(link)
    if image.size[1] > 1024:
        image = image.resize((image.size[0],1024))
    elif image.size[0] > 1024:
        image = image.resize((1024,image.size[1]))
    prompt = f"""{user_prompt}<|image_1|>/nYou are an Editor for a Magazine.
    Your Task is to describe and summarize the Image given above.
    Make sure to Include key features, entities and actions observed in the image.
    Stay true to the content in the Image and do not make up anything.
    Use the fact that this is an Image from the {link.split('/')[1]} as context to fill in any gaps in knowledge.
    {prompt_suffix}{assistant_prompt}"""
    inputs = processor(prompt,image,return_tensors = "pt").to("cuda:0")
    generation_args = { 
    "max_new_tokens": 425, 
    "temperature": 0.2, 
    "do_sample": True, 
    } 
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 
    return response

In [17]:
summarizeFigures("llamaParseDocs/"+pdfs[0]+"/"+pics[1])

(825, 613)


"The chart is a line graph representing the cumulative total return of various indices and a company from the year ended December 31, spanning from 2016 to 2021. The vertical axis is labeled 'Dollars' and ranges from $0 to $500 in increments of $50. The horizontal axis is labeled 'Year Ended December 31' and lists the years from 2016 to 2021. There are five lines, each with a unique marker, representing different indices and a company. The company, represented by a diamond marker, starts at $100 in 2016 and shows a steady increase to $445 in 2021. The Technology Index, marked by a triangle, begins at $100 in 2016 and rises to $358 in 2021. The Consumer Discretionary Index, indicated by a square, starts at $100 in 2016 and grows to $233 in 2021. The Consumer Staples Index, marked by a cross, begins at $100 in 2016 and increases to $327 in 2021. Lastly, the Consumer Cyclical Index, represented by a circle, starts at $100 in 2016 and reaches $445 in 2021. The chart is titled 'Cumulative T

In [43]:
for pdf in tqdm(pdfs[2:]):
    summaries = []
    pics = os.listdir("llamaParseDocs/"+pdf)
    for pic in tqdm(pics):
        if pic.endswith("jpg") == False:
            pass
        elif pic.startswith("table"):
            doc = Document(page_content = summarizeTables("llamaParseDocs/"+pdf+"/"+pic),metadata = {"source":"llamaParseDocs/"+pdf+"/"+pic})
            summaries.append(doc)
        elif pic.startswith("figure"):
            doc = Document(page_content = summarizeFigures("llamaParseDocs/"+pdf+"/"+pic),metadata = {"source":"llamaParseDocs/"+pdf+"/"+pic})
            summaries.append(doc)
    with open("llamaParseDocs/"+pdf+"/summaries"+".pkl", "wb") as f:
        pickle.dump(summaries, f)            

100%|██████████| 19/19 [15:31:53<00:00, 2942.83s/it]


In [6]:
import joblib
sums = joblib.load("Hackathon/llamaParseDocs/Apple_10-K-Q4-2023/summaries.pkl")

In [7]:
sums

[Document(page_content='The table presents data on the total fair value of an asset over different time periods. The total fair value is calculated by summing the values for each time period. The time periods considered are "Due after 1 year through 5 years", "Due after 5 years through 10 years", and "Due after 10 years". The total fair value is $100,544. The values for each time period are $74,427, $9,964, and $16,153 respectively. The total fair value is calculated by adding these three values together. The table provides a clear and concise representation of the total fair value of the asset over different time periods.', metadata={'source': 'llamaParseDocs/Apple_10-K-Q4-2023/table-40-23.jpg'}),
 Document(page_content='The table presents information about debt issuances and maturities for the years 2023 and 2022. It includes details such as the amount in millions, effective interest rate, and the calendar year for each debt issuance.\n\nFor the years 2023 and 2022, there are two typ